<h2>Rice Coding Compression<h2/>

<p>We need certain modules for the application to function, especially for wav/byte/bit manipulation.</p>

In [3]:
!pip install bitstring

import math
import wave

from bitstring import BitStream, BitArray

You should consider upgrading via the '/opt/conda/bin/python3 -m pip install --upgrade pip' command.


<h3>Encode</h3>

<p>The following code is the rice coding algorithm I made. It is used later for encoding the wav file.</p>

In [14]:
# unary function for use in encode
def unary(t):
    y=[];
    for i in range(t):
        y.append('1')
    y.append('0')
    return ''.join(y)

# rice encoding algorithm snippet
def r_enc(S, K):

    # find M
    M = 2 ** K

    # for S to be encoded, find:
    # quotient q = int(S/M)
    q = int(S//M)

    # remainder r = SmoduloM
    r = S%M
    
    # quotient code is q in unary
    q_un = unary(q)

    # remainder code is r in binary using K bits
    # func to return x in n-bit binary
    getbinary = lambda x, n: format(x, 'b').zfill(n)
    r_bin = getbinary(r, K)

    # codeword format <quotient code><remainder code>
    codeword = q_un + r_bin
    
    return codeword

<p>This code reads our wav file and returns a byte object.</p>

In [15]:
# getting binary data from wav file 
w = wave.open('Exercise2_Files/Sound1.wav', 'rb')
byte_data = w.readframes(w.getnframes())
w.close()

<p>We parse this byte object, encoding each byte (integer from 0-255) into bits as per the rice coding algorithm. For now, our results are stored as a list bits (each a result of r_enc).</p>

In [16]:
# bytes to bitstrings
bits = []
for byte in byte_data:
    bits.append(r_enc(byte, 4))

<p>Create an ex2 file to store our bits in. We use the join method to store one large bit string, separated by a space so we can split it later for decoding.</p>

In [17]:
# encode and write to ex2 file
with open("Exercise2_Files/Sound1_Enc.ex2", "w") as f:
    # Writing data to a file
    f.write(' '.join(bits))

<p>Encoding of the wav file and saving as an ex2 completed.</p>

<h3>Decode</h3>

<p>This is my decoder using the rice algorithm.</p>

In [22]:
# rice decoding algorithm snippet
def r_dec(codeword, K):
    cdw = list(codeword)
    
    # q by counting 1's before first 0
    q = 0
    
    for i in cdw:
        if i == '1':
            q += 1
        else:
            break
    
    # r reading next K bits as binary value
    r = ''.join(cdw[q:])
    r_int = int(r, 2)
    
    M = 2 ** K
    
    # S, encoded number, as q x M + r
    S = q * M + r_int
    
    return S

<p>Parsing the ex2 file using the split method, append each bit block to an array.</p>

In [23]:
bitsagain = []
    
with open("Exercise2_Files/Sound1_Enc.ex2", "r") as fd:
    bitsagain = fd.read().split(' ')

<p>Decode_blocklist takes this array, and a K value for the rice algorithm and returns a list of our original bytes.</p>

In [24]:
def decode_blocklist(blocklist, K):
    nums = []
        
    for block in blocklist:
        nums.append(r_dec(block, K))

    return nums

<p>Store our decoded list as a variable for later.</p>

In [25]:
# decoded list of integers from bitstring
decbits = decode_blocklist(bitsagain, 4)

<p>Conversion of this list back to a bytes object. We can check this to be true.</p>

In [26]:
# conversion of int list to bytes
bytesagain = bytes(decbits)
type(bytesagain)

bytes

<p>Let's confirm the equivalency of the original bytes object from reading our wav file, to the output of the r_dec -> bytes() process.</p>

In [27]:
# equivalency check of files
bytesagain == byte_data

True

<p>Let us then write the result back to a wav file, tagged with 'Dec'.</p>

In [14]:
with open('Exercise2_Files/Sound1_Enc_Dec.wav', mode='bx') as f:
    f.write(bytesagain)

<p>Observing both wav files in the directory, we also see they are identical in size.</p>

<p>Decoding and equivalency check complete.</p>

<h3>Finding the Optimal K</h3>

<p>We will attempt to find the optimal K. The steps and results are detailed in the report.</p>

<p>We see K = 7 can represent 255 in  bits, and 1 in 8. Let us select 7 as our K.</p>

In [58]:
len(r_enc(255, 7))

9

In [67]:
len(r_enc(1, 7))

8

<p>Open our Sound2.wav file as a byte object for compressing.</p>

In [62]:
# getting binary data from wav file 
w = wave.open('Exercise2_Files/Sound2.wav', 'rb')
byte_data2 = w.readframes(w.getnframes())
w.close()

<p>Convert our Sound1.wav bytes using r_enc.</p>

In [63]:
# bytes to bitstrings
bits2 = []
for byte in byte_data:
    bits2.append(r_enc(byte, 7))

<p>Convert our Sound2.wav bytes using r_enc.</p>

In [64]:
# bytes to bitstrings
bits3 = []
for byte in byte_data2:
    bits3.append(r_enc(byte, 7))

<p>Write results of Sound1.wav encode to file.</p>

In [65]:
# encode and write to ex2 file
with open("Exercise2_Files/Sound1_K7_Enc.ex2", "w") as f:
    # Writing data to a file
    f.write(' '.join(bits2))

<p>Write results of Sound2.wav encode to file.</p>

In [66]:
# encode and write to ex2 file
with open("Exercise2_Files/Sound2_K7_Enc.ex2", "w") as f:
    # Writing data to a file
    f.write(' '.join(bits3))

<p>The compression results can be seen in the report.</p>